<a href="https://colab.research.google.com/github/BoilerToad/BERT_basic/blob/main/Colab_BERT_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Header code for colab notebooks

In [2]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive
#drive.mount("/content/drive", force_remount=True)

/content/drive
Mounted at /content/drive


In [3]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_ROOT_PATH = "/content/drive/My Drive/Colab Notebooks"
# replace with your Github username 
GIT_USERNAME = "BoilerToad" 
# definitely replace with your
GIT_TOKEN = "1664e3d5c3bbf4630a6ca516b2473785c2d6842a"  
# Replace with your github repository in this case we want 
# to clone Fine-Tuning_BERT repository
GIT_REPOSITORY = "BERT_basic" 

MY_GOOGLE_DRIVE_PATH = join(MY_GOOGLE_DRIVE_ROOT_PATH, GIT_REPOSITORY)
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" # this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/My Drive/Colab Notebooks/BERT_basic
mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/BERT_basic’: File exists
GIT_PATH:  https://1664e3d5c3bbf4630a6ca516b2473785c2d6842a@github.com/BoilerToad/BERT_basic.git


In [4]:
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [5]:
! git clone "{GIT_PATH}"

fatal: destination path 'BERT_basic' already exists and is not an empty directory.


In [6]:
%cd "{PROJECT_PATH}"

%ls -l

/content/drive/My Drive/Colab Notebooks/BERT_basic
total 135
-rw------- 1 root root  6731 Oct 24 02:59 Colab_BERT_basic.ipynb
-rw------- 1 root root 95159 Oct 23 20:45 df_topic_key_point_argument_label_dev.parquet
-rw------- 1 root root 35149 Oct 24 02:59 LICENSE
-rw------- 1 root root    64 Oct 24 02:59 README.md


In [7]:
# Code for utilizing a GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [8]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 8.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.9 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
import torch
import scipy

from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

import numpy as np
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq

In [10]:
# Load BERT pre-trained model
model_BERT = BertModel.from_pretrained( 'bert-base-uncased', output_hidden_states = True )

# Put the model in evaluation mode aka feed-forward operation
model_BERT.eval()

# Load the BERT tokenizer
tokenizer_BERT = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Reference for the follwing example ... https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [11]:
#text = "Here is the sentence I want embeddings for."
text = "A real education is about giving students the tools to learn, think, and express themselves; dictating to them what to wear sends a strong message that we don't trust them to think on their own."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer_BERT.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer_BERT.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
a             1,037
real          2,613
education     2,495
is            2,003
about         2,055
giving        3,228
students      2,493
the           1,996
tools         5,906
to            2,000
learn         4,553
,             1,010
think         2,228
,             1,010
and           1,998
express       4,671
themselves    3,209
;             1,025
di            4,487
##cta        25,572
##ting        3,436
to            2,000
them          2,068
what          2,054
to            2,000
wear          4,929
sends        10,255
a             1,037
strong        2,844
message       4,471
that          2,008
we            2,057
don           2,123
'             1,005
t             1,056
trust         3,404
them          2,068
to            2,000
think         2,228
on            2,006
their         2,037
own           2,219
.             1,012
[SEP]           102


In [12]:
# As BERT is trained on sentence paris ... need to add 1's and 0's to distinguish which sentence each token belongs to 

#. In this case, we only have 1 sentence
segments_ids = [1] * len(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [13]:
# Run the text through BERT, and collect all of the hidden states produced from all 12 layers. 
with torch.no_grad():
  outputs = model_BERT(tokens_tensor, segments_tensors)

# Evaluating the model will return a different number of objects based on 
# how it's  configured in the `from_pretrained` call earlier. In this case, 
# becase we set `output_hidden_states = True`, the third item will be the 
# hidden states from all layers. See the documentation for more details:
# https://huggingface.co/transformers/model_doc/bert.html#bertmodel

  hidden_states = outputs[2]

In [14]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")

layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))

batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))

token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 45
Number of hidden units: 768


In [15]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 45, 768])


In [16]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 45, 768])

In [17]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 45, 768])

In [18]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([45, 13, 768])

In [19]:
# Stores the token vectors, with shape [45 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [45 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 45 x 3072


In [20]:
# Stores the token vectors, with shape [45 x 768]
token_vecs_sum = []

# `token_embeddings` is a [45 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 45 x 768


In [21]:
#  Average the second to last hidden layer of each token

# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


In [ ]:
table = pq.read_table( ('df_topic_key_point_argument_label_dev.parquet') )

df_topic_key_point_argument_label = table.to_pandas()

In [ ]:
df_topic_key_point_argument_label


In [ ]:
# list of topics from the dataframe
topic_list = df_topic_key_point_argument_label['topic'].unique().tolist()

topic_list

In [ ]:
df_topic_key_point_argument_label[(df_topic_key_point_argument_label['topic']=='We should abandon the use of school uniform') &
                                  (df_topic_key_point_argument_label['stance']==1)]['key_point'].unique().tolist()

In [ ]:
df_topic_key_point_argument_label[(df_topic_key_point_argument_label['topic']=='We should abandon the use of school uniform') &
                                  (df_topic_key_point_argument_label['stance']==1)]['argument'].unique().tolist()

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model_SBERT = SentenceTransformer('bert-base-nli-mean-tokens')
#model = SentenceTransformer('nli-deberta-base')

In [ ]:
# A corpus is a list with documents split by sentences.

sentences = ["School uniform is harming the student's self expression",
             'School uniforms are expensive',
             'School uniforms are often uncomfortable/sexist',
             'School uniform harms learning/creativity',
             'School uniforms increase conformity or harm individuality']


# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model_SBERT.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

#print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

In [ ]:
query = "A real education is about giving students the tools to learn, think, and express themselves; dictating to them what to wear sends a strong message that we don't trust them to think on their own." #@param {type: 'string'}

queries = [query]
query_embeddings = model_SBERT.encode(queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 5 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Argument:", query)
    print("\nTop 5 most similar key_points in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))